# The London Railway Network

The cell below defines the abstract class whose API you will need to impement. Do NOT modify it.

In [56]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

class AbstractLondonRailwayMapper(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # data initialisation
    @abstractmethod
    def loadStationsAndLines(self):
        pass

    # returns the minimum number of stops to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # numStops : int
    @abstractmethod
    def minStops(self, fromS, toS):     
        numStops = -1
        return numStops    
    
    # returns the minimum distance in miles to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # minDistance : float
    @abstractmethod
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        return minDistance
    
    # given an unordered list of station names, returns a new railway line 
    # (represented as a list of adjacent station names), connecting all such stations 
    # and such that the sum of the distances (in miles) between adjacent stations is minimised
    # inputList : set<str>
    # outputList : list<str>
    @abstractmethod
    def newRailwayLine(self, inputList):
        outputList = []
        return outputList

Use the cell below to define any data structure and auxiliary python function you may need. Leave the implementation of the main API to the next code cell instead.

In [57]:
# AVL Tree

class AVLTreeNode:

    def __init__(self, key, value):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1
        self.value = value

class AVLTree:

    def get(self, node, key):

        if node.key == key:
            return node
        elif key < node.key and node.left:
            return self.get(node.left, key)
        elif key > node.key and node.right:
            return self.get(node.right, key)
        else:
            return None

    def put(self, node, key, value):

        if (node is None):
            return AVLTreeNode(key, value)
        elif key < node.key:
            node.left = self.put(node.left, key, value)
        else:
            node.right = self.put(node.right, key, value)

        node.height = 1 + max(self.getHeight(node.left), self.getHeight(node.right))

        balance = self.getBalance(node)
        
        if balance > 1:
            if key < node.left.key:
                return self.rightRotate(node)
            else:
                node.left = self.leftRotate(node.left)
                return self.rightRotate(node)

        if balance < -1:
            if key > node.right.key:
                return self.leftRotate(node)
            else:
                node.right = self.rightRotate(node.right)
                return self.leftRotate(node)

        return node

    def leftRotate(self, node):
        savedRight = node.right
        savedRightLeft = savedRight.left
        savedRight.left = node
        node.right = savedRightLeft
        node.height = 1 + max(self.getHeight(node.left),
                           self.getHeight(node.right))
        savedRight.height = 1 + max(self.getHeight(savedRight.left),
                           self.getHeight(savedRight.right))
        return savedRight

    def rightRotate(self, node):
        savedLeft = node.left
        savedLeftRight = savedLeft.right
        savedLeft.right = node
        node.left = savedLeftRight
        node.height = 1 + max(self.getHeight(node.left),
                           self.getHeight(node.right))
        savedLeft.height = 1 + max(self.getHeight(savedLeft.left),
                           self.getHeight(savedLeft.right))
        return savedLeft


    def getHeight(self, node):
        if node is None:
            return 0
        return node.height

    def getBalance(self, node):
        if node is None:
            return 0
        return self.getHeight(node.left) - self.getHeight(node.right)



In [58]:
# Queue

class QueueNode:
    def __init__(self, data = None):
        self.data = data
        self.next = None
        self.prev = None

class MyQueue:

    def __init__(self):
        self.size = 0
        self.head = None
        self.tail = None

    def enqueue(self, data):
        node = QueueNode(data)
        if (self.head == None):
            self.head = node
            self.tail = node
        else:
            self.tail.next = node
            node.prev = self.tail
            self.tail = node
        self.size += 1

    def dequeue(self):
        data = self.head.data
        self.head = self.head.next
        self.size -= 1
        if (self.size == 0):
            self.tail = self.head
        return data

    def isEmpty(self):
        return self.size == 0



In [59]:
import csv
import math

# ADD YOUR DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE



class DataLoader:

    def __init__(self):
        self.stationsTree = AVLTree()
        self.root = None
        self.stationsCount = -1
        self.graph = None

    def distance(self, longitude1, latitude1, longitude2, latitude2):
        distance = math.sqrt( (longitude1 - longitude2)**2 + (latitude1 - latitude1)**2 )
        return distance

    def loadFiles(self):

        with open('londonstations.csv') as stationsFile:
            reader = csv.reader(stationsFile, delimiter=',')
            for row in reader:
                if (self.stationsCount == -1):
                    self.stationsCount += 1
                    continue
                self.root = self.stationsTree.put(self.root, row[0], [self.stationsCount, float(row[1]), float(row[2])])
                self.stationsCount += 1

        self.graph = WeightedGraph(self.stationsCount)
        print(self.graph)

        with open('londonrailwaylines.csv') as linesFile:
            reader = csv.reader(linesFile, delimiter=',')
            lineCount = 0
            for row in reader:
                if (lineCount == 0): 
                    lineCount += 1
                    continue
                lineName = row[0]
                fromStation = row[1]
                toStation = row[2]
                fromStationNode = self.stationsTree.get(self.root, fromStation)
                toStationNode = self.stationsTree.get(self.root, toStation)

                dist = self.distance(fromStationNode.value[2], fromStationNode.value[1], toStationNode.value[2], toStationNode.value[1])
                edge = Edge(fromStationNode.value[0], toStationNode.value[0], dist, lineName)
                self.graph.addEdge(edge)
                lineCount += 1

        return self.graph, self.stationsTree, self.root



class WeightedGraph:

    def __init__(self, V):
        self.V = V
        self.adj = []
        for _ in range (0, V):
            self.adj.append([])
        print(self.adj)

    def addEdge(self, e):
        v = e.endPoint()
        w = e.otherEndPoint(v)
        self.adj[v].append(e)
        self.adj[w].append(e)

    def adjList(self, v):
        return self.adj[v]

    def getVertexCount(self):
        return self.V

class Edge:

    def __init__(self, v, w, weight, lineName):
        self.v = v
        self.w = w
        self.weight = weight
        self.lineName = lineName

    def getLineName(self):
        return self.lineName

    def endPoint(self):
        return self.v

    def otherEndPoint(self, vertex):
        if vertex == self.v: return self.w
        else: return self.v

    def compareTo(self, edge):
        if (self.weight < edge.weight): return -1
        elif (self.weight < edge.weight): return +1
        else: return 0

class MinStopsBFS:

    def __init__(self, graph):
        self.graph = graph
        self.distToSource = [-1] * self.graph.getVertexCount()
    
    def minStops(self, start, end):
        queue = MyQueue()    # Implement a queue
        queue.enqueue(start)

        self.distToSource[start] = 0

        while (not queue.isEmpty()):
            v = queue.dequeue()
            print(self.graph.adjList(v))
            for edge in self.graph.adjList(v):
                w = edge.otherEndPoint(v)
                if (self.distToSource[w] == -1):
                    queue.enqueue(w)
                    self.distToSource[w] = self.distToSource[v] + 1
        return self.distToSource[end]






In [60]:
import csv

class LondonRailwayMapper(AbstractLondonRailwayMapper):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        self.graph = None
        self.stations = None
        self.rootStationsTree = None

        pass           
     
    
        
    def loadStationsAndLines(self):
        # ADD YOUR CODE HERE
        print("here1")
        loader = DataLoader()
        self.graph, self.stations, self.rootStationsTree = loader.loadFiles()
        print("Graph: ", self.graph)
        print("here2")
    
    

    def minStops(self, fromS, toS):     
        numStops = -1
        # ADD YOUR CODE HERE
        print("Graph: ", self.graph)
        finder = MinStopsBFS(self.graph)
        numStops = finder.minStops(self.stations.get(self.rootStationsTree, fromS).value[0], self.stations.get(self.rootStationsTree, toS).value[0])
        
        return numStops    
    
    
    
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        # ADD YOUR CODE HERE
        # In order to implement this requirement I decided to use the Dijkstra (binary heap) 
        # algorithm. This because it is O(E * logV) in typical 

        
        return minDistance
    
    
    
    
    def newRailwayLine(self, inputList):
        outputList = []
        # ADD YOUR CODE HERE

        
        return outputList

Use the cell below for all python code needed to test the `LondonRailwayMapper` class above.

In [61]:
import timeit

# ADD YOUR TEST CODE HERE

mapper = LondonRailwayMapper()
mapper.loadStationsAndLines()
stops = mapper.minStops("Green Park", "Covent Garden")
print("done, stops: ", stops)



[<__main__.Edge object at 0x000001C9959E80A0>, <__main__.Edge object at 0x000001C9959E8A60>]
[<__main__.Edge object at 0x000001C9959E8100>, <__main__.Edge object at 0x000001C9959E8A60>]
[<__main__.Edge object at 0x000001C992D07E80>, <__main__.Edge object at 0x000001C992D07F10>]
[<__main__.Edge object at 0x000001C992D07250>, <__main__.Edge object at 0x000001C992D00700>]
[<__main__.Edge object at 0x000001C992D07B20>, <__main__.Edge object at 0x000001C992D07B50>]
[<__main__.Edge object at 0x000001C992D07940>, <__main__.Edge object at 0x000001C992D07FD0>]
[<__main__.Edge object at 0x000001C994FB0DF0>, <__main__.Edge object at 0x000001C994FA1AF0>, <__main__.Edge object at 0x000001C992CAFD00>, <__main__.Edge object at 0x000001C992CAFF40>]
[<__main__.Edge object at 0x000001C995357130>, <__main__.Edge object at 0x000001C995357220>]
[<__main__.Edge object at 0x000001C992D075B0>, <__main__.Edge object at 0x000001C992D07E50>]
[<__main__.Edge object at 0x000001C992D073D0>, <__main__.Edge object at

The cell below exemplifies the test code I will invoke on your submission. Do NOT modify it. 

In [62]:
# DO NOT MODIFY THIS CELL

import timeit

testMapper = LondonRailwayMapper()

#
# testing the loadStationsAndLines() API 
#
starttime = timeit.default_timer()
testMapper.loadStationsAndLines()
endtime = timeit.default_timer()
print("\nExecution time to load:", round(endtime-starttime,3))

#
# testing the minStops() and minStops() API on a sample of from/to station pairs  
#
fromList = ["Baker Street", "Epping", "Canonbury", "Vauxhall"]
toList = ["North Wembley", "Belsize Park", "Balham", "Leytonstone"]

for i in range(len(fromList)):
    starttime = timeit.default_timer()
    stops = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("\nExecution time minStops:", round(endtime-starttime,3))

    starttime = timeit.default_timer()
    dist = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("Execution time minDistance:", round(endtime-starttime,3))

    print("From", fromList[i], "to", toList[i], "in", stops, "stops and", dist, "miles")  
    
#
# testing the newRailwayLine() API on a small list of stations  
#
stationsList = ["Queens Park", "Chigwell", "Moorgate", "Swiss Cottage", "Liverpool Street", "Highgate"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))

#
# testing the newRailwayLine() API on a big list of stations  
#
stationsList = ["Abbey Road", "Barbican", "Bethnal Green", "Cambridge Heath", "Covent Garden", "Dollis Hill", "East Finchley", "Finchley Road and Frognal", "Great Portland Street", "Hackney Wick", "Isleworth", "Kentish Town West", "Leyton", "Marble Arch", "North Wembley", "Old Street", "Pimlico", "Queens Park", "Richmond", "Shepherds Bush", "Tottenham Hale", "Uxbridge", "Vauxhall", "Wapping"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))

ain__.Edge object at 0x000001C9951CC490>, <__main__.Edge object at 0x000001C9951C7130>, <__main__.Edge object at 0x000001C9951C7EB0>]
[<__main__.Edge object at 0x000001C9951C93D0>, <__main__.Edge object at 0x000001C9951C9F10>]
[<__main__.Edge object at 0x000001C9950EE130>, <__main__.Edge object at 0x000001C9950CE4F0>]
[<__main__.Edge object at 0x000001C9950EE490>, <__main__.Edge object at 0x000001C9950EE850>]
[<__main__.Edge object at 0x000001C9950EE430>, <__main__.Edge object at 0x000001C9950EE610>]
[<__main__.Edge object at 0x000001C9950EE670>, <__main__.Edge object at 0x000001C9950CE130>]
[<__main__.Edge object at 0x000001C9950EE910>, <__main__.Edge object at 0x000001C9950EEC10>]
[<__main__.Edge object at 0x000001C9950EE070>, <__main__.Edge object at 0x000001C9950EEB50>]
[<__main__.Edge object at 0x000001C9950CE970>, <__main__.Edge object at 0x000001C9951CC1F0>]
[<__main__.Edge object at 0x000001C9951C9F70>, <__main__.Edge object at 0x000001C9951C9FD0>]
[<__main__.Edge object at 0x0

IndexError: list index out of range